In [ ]:
%load_ext autoreload

import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')


import numpy as np
import numpy.linalg as la
import scipy as sp
import scipy.io as spio

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib as mpl
import seaborn as sns
import matplotlib.colors as mcolors


import itertools as it
import queue
import networkx as nx
import skimage.morphology as morph

import homology

mpl.rcParams['mathtext.fontset'] = 'cm'
sns.set_context('poster', font_scale=1.25)
sns.set(color_codes=True, palette='deep')
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})

In [ ]:
mat = spio.loadmat("../sample_data/Z17.mat")
data = mat['Z17'][2400:2475, 2400:2475][52:58, 3:8]
# data = mat['Z17'][1500:2000, 1500:2000]

# mat = spio.loadmat("../sample_data/thresholded/Creases15.mat")
# data = mat['ridges'][1750:2000, 1750:2000].astype(np.int8)

print(data.shape)

# print(np.min(data), np.max(data))


# data = np.array([[7, 6, 5, 4],
#                 [8, 5, 4, 3],                        
    
#                 [9, 4, 3, 2],
#                 [0, 3, 2, 1]])

# data = np.array([[3,2,2],
#                 [3,1,1],
#                 [0,1,0]])

# data = np.array([[8, 3, 2],
#                 [7, 8, 1],
#                 [6, 8, 1]
#                 [5, 10, 0]])

# # data = mat['ridges'][1000:2000, 1000:2000].astype(np.int8)+mat['valleys'][1000:2000, 1000:2000].astype(np.int8)

# # data = mat['ridges'].astype(np.int8)

fig, ax = plt.subplots(figsize=(16,16))
im = ax.imshow(data, cmap=plt.cm.Blues_r)
ax.axis('off')
# plt.colorbar(im)
plt.show()

In [ ]:
%autoreload

print("Constructing complex")
comp = homology.construct_cubical_complex(data.shape)

print("Constructing cofacets")
comp.construct_cofacets()

print("Constructing discrete gradient")

# heights = np.argsort(np.argsort(data.flatten()))
heights = data.flatten()

V = homology.construct_discrete_gradient(comp, heights)

# print(V)

# for v in V:
#     if v == V[v]:
#         print(v, comp.dims[v])
   
print("Reversing gradient")
coV = homology.reverse_discrete_gradient(V)

print("Finding basins")
basins = homology.find_basins(coV, comp.cofacets, comp.dims, 0)
    
print("Calculating Morse skeleton")
skeleton = homology.find_morse_skeleton(V, coV, comp.facets, comp.cofacets, comp.dims, 1)


print("Calculating Morse complex")
mcomp = homology.construct_morse_complex(V, comp.facets, comp)

print(mcomp.facets)
print(mcomp.dims)

print("Calculating persistence pairs...")

filtration = homology.construct_filtration(comp.facets, comp.dims, mcomp, heights)

(pairs, cell_index) = homology.compute_persistence_pairs(mcomp, filtration, show_zero=True)

print(pairs)

print("Complete")

In [ ]:
palette = it.cycle(sns.color_palette("deep"))
# palette = it.cycle(sns.color_palette("Greens"))

image = np.ones([data.shape[0]*data.shape[1], 3])

for i in basins:
        
#     if i == 0:
# #         image[list(basins[i])] = (0.0, 0.0, 0.0)
#     else:
#         if len(segments[i]) == 0:
#             print(i)

#     print(basins[i])

    color = next(palette)
    image[list(basins[i])] = color
    
    
# image[list(skeleton)] = (0.0, 0.0, 0.0)

# for i in basins:
#     image[i] = (1.0, 1.0, 1.0)
        
print(np.min(data), np.max(data))
        
# image[np.where(np.argsort(np.argsort(data.flatten())) > 0 )[0]] = (1.0, 1.0, 1.0)


# print(np.argsort(np.argsort(data.flatten())))

fig, ax = plt.subplots(figsize=(32, 32))
ax.imshow(image.reshape((data.shape[0], data.shape[1], 3)))


ax.axis('off')
plt.show()

In [ ]:
birth = [[] for i in range(3)]
death = [[] for i in range(3)]
mult = [[] for i in range(3)]
for d in range(3):
    for (i, j) in list(ipairs[d].keys()):
        birth[d].append(hbins[i])
        death[d].append(hbins[j])
        mult[d].append(ipairs[d][(i, j)])

pbirth = [[] for i in range(3)]
for d in range(3):
    for i in (persist[d].keys()):
        pbirth[d].append(hbins[i])

# print(birth)
# print(death)

for d in range(3):
    
    fig = plt.figure(figsize=(8,8))
    
    ax1 = fig.add_subplot(1,1,1)

    ax1.scatter(birth[d], death[d], c=mult[d], marker='.', cmap='Blues', edgecolors='k', linewidths=1.0, s=100)
#     ax1.scatter(birth[d], death[d], c='b', marker='.', s=20)
    
    ax1.scatter(pbirth[d], pbirth[d], marker='s', color='r')

#     ax1.plot(np.linspace(0, 125, 100), np.linspace(0, 125, 100), 'k--')
    
    ax1.plot(np.linspace(np.min(heights), np.max(heights), 100), np.linspace(np.min(heights), np.max(heights), 100), 'k--')
    
    ax1.set_title(r"$d={}$".format(d))

#     ax1.set_xscale('symlog', linthreshx=1e-16)
#     ax1.set_yscale('symlog', linthreshy=1e-16)
    
    ax1.set_xlabel(r"birth")
    ax1.set_ylabel(r"death")
        
    plt.tight_layout()

    plt.show()


In [ ]:
for d in range(2):

    fig = plt.figure(figsize=(8,8))
    
    ax1 = fig.add_subplot(1,1,1)

    ax1.scatter(birth[d], np.array(death[d])-np.array(birth[d]), c=mult[d], marker='.', cmap='Blues', edgecolors='k', linewidths=1.0, s=10)
    
    ax1.scatter(pbirth[d], np.zeros_like(pbirth[d]), marker='s', color='r')
        
    ax1.set_title(r"$d={}$".format(d))


    ax1.set_yscale('log')
    
    ax1.set_ylim(np.max([np.min(np.array(death[d])-np.array(birth[d]))/2.0, 1e-8]), 
                 np.max(np.array(death[d])-np.array(birth[d]))*2.0)
    
    print(np.min(np.array(death[d])-np.array(birth[d])), np.max(np.array(death[d])-np.array(birth[d])))
    
    ax1.set_xlabel(r"birth")
    ax1.set_ylabel(r"persistence")
        
    plt.tight_layout()

    plt.show()

In [ ]:
%autoreload
# G, positions = homology.construct_mesh_graph(data.shape[0], data.shape[1], diagonals=True, pos=True)

# segments = homology.compute_graph_segmentation(G, data.flatten(), True, positions)

print("Constructing Mesh...")

G, pos = homology.construct_mesh_graph(data.shape[0], data.shape[1], triangulate=True, get_pos=True)

nx.draw(G, pos={node: (x,-y) for (node, (x,y)) in pos.items()})
plt.show()

print("Segmenting Graph...")

segments = homology.compute_graph_segmentation(G, data.flatten(), False, None)


In [ ]:
%autoreload


def construct_subcomplex(comp, verts):
    
    sub_comp = homology.CellComplex(comp.dim)
    
    for vi in verts:
        
        sub_comp.add(0, vi, [])
        
    for d in range(1, comp.dim+1):
        
        for fi in comp.faces[d]:
            bound = []
            complete = True
            for bi in comp.faces[d][fi]:
                if bi in sub_comp.faces[d-1]:
                    bound.append(bi)
                else:
                    complete = False
                    break
                    
            if complete:
                
                sub_comp.add(d, fi, bound)
    return sub_comp


def lower_star_filtration(comp, vheights):
    
    heights = [{} for d in range(comp.dim+1)]
    
    for vi in comp.faces[0]:
        heights[0][vi] = vheights[vi]
        
    for d in range(1, comp.dim+1):
        for ci in comp.faces[d]:
            
            bheights = []
            for cj in comp.faces[d][ci]:
                bheights.append(heights[d-1][cj])
            
            heights[d][ci] = np.max(bheights)
            
     
    cells = []
    dims = []
    height_list = []
    
    for d in range(comp.dim+1):
        for ci in heights[d]:
            cells.append(ci)
            dims.append(d)
            height_list.append(heights[d][ci])
    
        
    return (cells, dims, height_list)
    
    

def get_cofaces(comp):
    
    cofaces = [{} for i in range(comp.dim)]
    for d in range(comp.dim):
        for cell in comp.faces[d]:
            cofaces[d][cell] = []    
    
    for d in range(1, comp.dim+1):
        for icoface in comp.faces[d]:
            for iface in comp.faces[d][icoface]:
                cofaces[d-1][iface].append(icoface)
                
    return cofaces

def add_segment_faces(comp, G, segments):
    
    cofaces = get_cofaces(comp)
    
    for i in segments:
        if i == 0:
            continue
          
        marked = [set() for d in range(comp.dim)]  
        
        cycle = []
        bound = set()
        
        for vi in segments[i]:
            
            for nbr in G[vi]:
                if nbr in segments[0] and nbr not in marked[0]:
                    marked[0].add(nbr)
                    
                   
                    for ei in cofaces[0][nbr]:
                        include = True
                        
                        for vj in comp.faces[1][ei]:
                            if vj not in marked[0]:
                                include = False
                                break
                            
                        if include:
                            cycle.append(ei)
                            bound ^= set(comp.faces[1][ei])
                            
             
        if len(bound) == 0:
            comp.add(2, -i, cycle)                       
                
def get_segment_heights(segments, vheights, comp):
    
    heights = {}
    
    for i in segments:
        
        
        if -i not in comp.faces[2]:
            continue
            
        bheights = []
        for vi in segments[i]:
            bheights.append(vheights[vi])
          
        heights[-i] = np.max(bheights)
        
        if i == 95:
            print(bheights)
        
    return heights
            
            
        

comp = homology.construct_mesh_complex(data.shape[0], data.shape[1])

wshed = np.array(list(segments[0]))

sub_comp = construct_subcomplex(comp, wshed)

(cells, dims, heights) = lower_star_filtration(sub_comp, -data.flatten())

add_segment_faces(sub_comp, G, segments)

seg_heights = get_segment_heights(segments, -data.flatten(), sub_comp)

for si in seg_heights:
    cells.append(si)
    dims.append(2)
    heights.append(seg_heights[si])


(ipairs, hbins, persist, cell_to_pindex) = homology.compute_persistence_pairs(sub_comp, cells, dims, heights)


In [ ]:
image = np.ones([data.shape[0]*data.shape[1], 3])
    
image[list(segments[0])] = (0.0, 0.0, 0.0)

palette = it.cycle(sns.color_palette("deep"))

for vi in range(G.order()):
    
    is_min = True
    for nbr in G[vi]:
        if data.flatten()[nbr] < data.flatten()[vi]:
            is_min = False
            
    if is_min:
        image[vi] = next(palette)

fig, ax = plt.subplots(figsize=(16, 16))
ax.imshow(image.reshape((data.shape[0], data.shape[1], 3)))


ax.axis('off')
plt.show()

In [ ]:
palette = it.cycle(sns.color_palette("deep"))

for i in sub_comp.faces[2]:
    
    if i != -3:
        continue
    
    print(i)
    
    pixels = set()
    for ei in sub_comp.faces[2][i]:
        pixels.update(sub_comp.faces[1][ei])

    image = np.ones([data.shape[0]*data.shape[1], 3])
    
    image[list(segments[0])] = (0.0, 0.0, 0.0)
    image[list(pixels)] = next(palette)
    
    print(data.flatten()[sorted(list(pixels))])
    print(data.flatten()[sorted(list(segments[-i]))])
    
    print(pixels)
    print(segments[-i])
    
    
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(image.reshape((data.shape[0], data.shape[1], 3)))


    ax.axis('off')
    plt.show()



In [ ]:
G, positions = homology.construct_mesh_graph(data.shape[0], data.shape[1], diagonals=True, pos=True)

                                             
dist = homology.compute_graph_dilation(G, np.where(data.flatten() == 1)[0], euclidean=True, positions=positions)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))    
ax.imshow(dist.reshape(data.shape), cmap=plt.cm.Blues_r)
ax.axis('off')
plt.show()

In [ ]:
%autoreload

segments = homology.compute_graph_segmentation(G, -dist)

In [ ]:
palette = it.cycle(sns.color_palette("deep", 6))

image = np.zeros([data.shape[0]*data.shape[1], 3])

for i in segments:
    if i == 0:
        image[list(segments[i])] = (0.0, 0.0, 0.0)
    else:
        color = next(palette)
        image[list(segments[i])] = color

# image[np.where(data.flatten() == 1)[0]] = (0.0, 0.0, 0.0)

# image[np.where(dist > 27 )[0]] = (1.0, 1.0, 1.0)

fig, ax = plt.subplots(figsize=(32,32))
ax.imshow(image.reshape((data.shape[0], data.shape[1], 3)), cmap=plt.cm.gray)
ax.axis('off')
plt.show()

In [ ]:
%autoreload

heights = data.flatten()

print("Creating Mesh")
comp = homology.construct_mesh_complex(data.shape[0], data.shape[1], False)


# print(data.shape)

# print(comp.faces)

# heights = np.append(heights, np.min(heights)-1)

print("Creating Filtration")
(simp_filt, dims) = homology.construct_lower_star_filtration(comp, np.argsort(heights))


# print(simp_filt)
# print(dims)

print("Calculating Persistence")
(ipairs, hsort, persist, sim_to_pindex) = homology.compute_persistence_pairs(simp_filt, dims, sorted(heights), comp)

print("Persistence Pairs:", ipairs)
print("Infinite Persistence:", persist)

In [ ]:
%autoreload

G, pos = homology.construct_mesh_graph(3, 3, get_pos=True)

nx.draw(G, pos=pos)

plt.show()

comp = homology.construct_mesh_complex(3, 3)

print(comp.faces)